In [1]:
import numpy as np
import tensorflow as tf


# Step 1: Generate example data
num_train_examples = 20000
sequence_length = 8

batch_size = 64
num_epochs = 25
val_split = 0.2

rng = np.random.default_rng(2024)

In [2]:
# Generate random frequencies for sine waves
# we'll use 1.0 as the maximum Nyquist frequency and 0 = constant
frequencies = rng.uniform(0.02, 0.2, size=num_train_examples)
phase_offsets = rng.uniform(0.0, 2*np.pi, size=num_train_examples)
sequences = np.zeros((num_train_examples, sequence_length))

# Generate sine waves
for i in range(num_train_examples):
    sequences[i] = np.sin(2*np.pi*frequencies[i]* np.arange(sequence_length) + phase_offsets[i])

# Split sequences into input (first <sequence_length-1> elements) and labels (sequence_length>'th element)
x_train = sequences[:, :sequence_length-1]
y_train = sequences[:, sequence_length-1]






In [3]:
from tensorflow.keras import models, layers

# Build the model
model = models.Sequential([
  layers.InputLayer(input_shape=x_train.shape[1:]), 
  layers.Dense(64, activation='relu'),
  layers.BatchNormalization(),
  layers.Dense(64, activation='relu'),
  layers.BatchNormalization(),
  layers.Dense(1)  # Single neuron for regression
])
input_shape = x_train.shape[1:]
print(input_shape)

# Compile the model
steps_per_epoch = int((1.0-val_split)*num_train_examples / batch_size)
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(1e-2, steps_per_epoch*num_epochs)
  
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), # use tf.keras.optimizers.Adam on Intel / NVidia GPU
              loss='mean_squared_error'
             )

(7,)


c:\Users\super\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [4]:
# Train the model
train_hist = model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=val_split)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2979 - val_loss: 0.1519
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0182 - val_loss: 0.0328
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0173 - val_loss: 0.0231
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0162 - val_loss: 0.0019
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0153 - val_loss: 0.0041
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0125 - val_loss: 0.0028
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0112 - val_loss: 0.0014
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0116 - val_loss: 0.0033
Epoch 9/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0115 - val_loss: 0.0031
Epoch 10/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0114 - val_loss: 0.0014
Epoch 11/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0106 - val_loss: 0.0013
Epoch 12/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [5]:
print(f"Using TF Version: {tf.version.VERSION}")

choice = x_train[np.random.randint(0, x_train.shape[0])]
print(f"Input: {choice}")

def representative_dataset():
    # Generate synthetic sine wave data matching training distribution
    for _ in range(100):
        yield [x_train[np.random.randint(0, x_train.shape[0])].astype(np.float32)]
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

with open('new_sin_predictor.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model saved as new_sin_predictor.tflite")


Using TF Version: 2.18.0
Input: [-0.0064896  -0.87153519 -0.85803593  0.02040035  0.8782722   0.850808
 -0.03430716]
INFO:tensorflow:Assets written to: C:\Users\super\AppData\Local\Temp\tmpzef4jtdt\assets


INFO:tensorflow:Assets written to: C:\Users\super\AppData\Local\Temp\tmpzef4jtdt\assets


Saved artifact at 'C:\Users\super\AppData\Local\Temp\tmpzef4jtdt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2628912920912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931292496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931300768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931302352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931296192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931298480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931303584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931304288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931388688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931304640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2628931300240: 

c:\Users\super\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\lite\python\convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Model saved as new_sin_predictor.tflite


In [6]:
# Test in Python
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get quantization parameters
input_scale = input_details[0]['quantization'][0]
input_zero_point = input_details[0]['quantization'][1]

print(f"Input Scale: {input_scale} | Input Zero Point: {input_zero_point}")
print(f"Output Scale: {output_details[0]['quantization'][0]} | Output Zero Point: {output_details[0]['quantization'][1]}")

# Scale and quantize input
raw_input = x_train[0].reshape(1, -1)
quantized_input = raw_input / input_scale + input_zero_point
input_data = quantized_input.astype(np.int8)
print("Raw Input:", raw_input)
print("Quantized Input:", input_data)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Quantized Prediction:", output)

# Dequantize output
output_scale = output_details[0]['quantization'][0]
output_zero_point = output_details[0]['quantization'][1]
dequantized_output = (output.astype(np.float32) - output_zero_point) * output_scale
print("Dequantized Prediction:", dequantized_output)

size = 0
for tensor in interpreter.get_tensor_details():
    match tensor['dtype']:
        case np.int8:
            size += np.prod(tensor['shape'])
        case np.float32:
            size += np.prod(tensor['shape']) * 4
        case np.int32:
            size += np.prod(tensor['shape']) * 4
        case _:
            raise ValueError(f"Unsupported data type: {tensor['dtype']}")
    
            

print(f"Arena Size: {size} bytes")

Input Scale: 0.007843123748898506 | Input Zero Point: 0
Output Scale: 0.007811177987605333 | Output Zero Point: 0
Raw Input: [[ 0.86234552  0.93621396  0.31616721 -0.53822045 -0.9936829  -0.7126346
   0.09661345]]
Quantized Input: [[ 109  119   40  -68 -126  -90   12]]
Quantized Prediction: [[104]]
Dequantized Prediction: [[0.8123625]]
Arena Size: 5260 bytes


In [7]:
import xxd
xxd.binary_to_c_array('new_sin_predictor.tflite', 'new_sin_predictor_data.h')